## Full Run

### In order to run the scripts, we need to be in the base directory. This will move us out of the notebooks directory and into the base directory

In [1]:
import os
os.chdir('..')

### Define where each of the datasets are stored

In [2]:
Xtrain_dir = 'solar/data/kaggle_solar/train/'
Xtest_dir = 'solar/data/kaggle_solar/test'
ytrain_file = 'solar/data/kaggle_solar/train.csv'
station_file = 'solar/data/kaggle_solar/station_info.csv'
import numpy as np

### Define the parameters needed to run the analysis script.

In [3]:
# Choose up to 98 stations; not specifying a station means to use all that fall within the given lats and longs. If the
# parameter 'all' is given, then it will use all stations no matter the provided lats and longs
station = ['all']

# Determine which dates will be used to train the model. No specified date means use the entire set from 1994-01-01 
# until 2007-12-31.
train_dates = ['1994-01-01', '2007-12-31']

#2008-01-01 until 2012-11-30
test_dates = ['2008-01-01', '2012-11-30']

station_layout = True

# Use all variables
var = ['all']

# Keep model 0 (the default model) as a column for each of the variables (aggregated over other dimensions)
model = [0]

# Aggregate over all times
times = ['all']

default_grid = {'type':'relative', 'axes':{'var':var, 'models':model, 'times':times,
                                          'station':station}}
# This just uses the station_names as another feature
stat_names = {'type':'station_names'}

frac_dist = {'type':'frac_dist'}

days_solstice = {'type':'days_from_solstice'}
days_cold = {'type':'days_from_coldest'}

all_feats = [stat_names, default_grid, frac_dist, days_solstice, days_cold]
#all_feats = [stat_names, days_solstice, days_cold]

### Define the directories that contain the code needed to run the analysis

In [4]:
import solar.report.submission
import solar.wrangle.wrangle
import solar.wrangle.subset
import solar.wrangle.engineer
import solar.analyze.model

### Reload the modules to load in any code changes since the last run. Load in all of the data needed for the run and store in a pickle file. The 'external' flag determines whether to look to save the pickle file in a connected hard drive or to store locally. The information in pink shows what has been written to the log file.

In [ ]:
# test combination of station names and grid
reload(solar.wrangle.wrangle)
reload(solar.wrangle.subset)
reload(solar.wrangle.engineer)
from solar.wrangle.wrangle import SolarData

#external = True

input_data = SolarData.load(Xtrain_dir, ytrain_file, Xtest_dir, station_file, \
                                  train_dates, test_dates, station, \
                                  station_layout, all_feats, 'extern')

INFO:solar.wrangle.wrangle:Started building test and training data
INFO:solar.wrangle.wrangle:Features: [{'type': 'station_names'}, {'axes': {'var': ['all'], 'models': [0], 'station': ['all'], 'times': ['all']}, 'type': 'relative'}, {'type': 'frac_dist'}, {'type': 'days_from_solstice'}, {'type': 'days_from_coldest'}]
INFO:solar.wrangle.wrangle:Train dates: ['1994-01-01', '2007-12-31']
INFO:solar.wrangle.wrangle:Test dates: ['2008-01-01', '2012-11-30']
INFO:solar.wrangle.wrangle:Stations: ['all']


In [ ]:
reload(solar.analyze.model)
import numpy as np
from solar.analyze.model import Model

from sklearn.linear_model import Ridge
from sklearn import metrics

error_formula = 'mean_absolute_error'

model = Model.model(input_data, Ridge, {'alpha':np.logspace(-3,1,10,base=10)}, 10, 
                    error_formula, 4, 'extern', normalize=True)

In [ ]:
reload(solar.analyze.model)
import numpy as np
from solar.analyze.model import Model

from sklearn.ensemble import GradientBoostingRegressor
from sklearn import metrics

error_formula = 'mean_absolute_error'

model = Model.model_from_pickle('input_2016-02-06-18-17-28.p', GradientBoostingRegressor, {'n_estimators':range(100,500, 100), 'learning_rate':np.logspace(-3,1,5,base=10)}, 10, 
                    error_formula, loss='ls', max_depth=1, random_state=0)

In [ ]:
reload(solar.report.submission)
from solar.report.submission import Submission

preds = Submission.submit_from_pickle('model_2016-02-06-18-21-41.p', 'input_2016-02-06-18-17-28.p', True)

